In [1]:
from dask_jobqueue import SLURMCluster 
from dask.distributed import Client 
  
cluster = SLURMCluster(cores=28,name='make_profiles',walltime='00:30:00',job_extra=['--constraint=HSW24','--exclusive','--nodes=1'],memory='120GB',interface='ib0') 
cluster.scale(140)
cluster

/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/distributed/bokeh/core.py:74: UserWarning: 
Failed to start diagnostics server on port 8787. [Errno 13] Permission denied
  warnings.warn("\n" + msg)


In [2]:
from dask.distributed import Client
client = Client(cluster)
client

Client Scheduler: tcp://172.30.100.1:37497 Dashboard: http://172.30.100.1:46096/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [3]:
!squeue -u albert7a

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
           8152699     hsw24 make_pro albert7a PD       0:00      1 (Resources)
           8152700     hsw24 make_pro albert7a PD       0:00      1 (Priority)
           8152701     hsw24 make_pro albert7a PD       0:00      1 (Priority)
           8152702     hsw24 make_pro albert7a PD       0:00      1 (Priority)
           8152703     hsw24 make_pro albert7a PD       0:00      1 (Priority)
           8152431     hsw24 make_pro albert7a  R      17:58      1 n2160
           8152432     hsw24 make_pro albert7a  R      17:58      1 n2829
           8152433     hsw24 make_pro albert7a  R      17:58      1 n2831
           8152434     hsw24 make_pro albert7a  R      17:58      1 n2832
           8152435     hsw24 make_pro albert7a  R      17:58      1 n2833
           8152436     hsw24 make_pro albert7a  R      17:58      1 n2835
           8152437     hsw24 make_pro albert7a  R      17:58      1 n2209


In [4]:
import time
nb_workers = 0
while True:
    nb_workers = len(client.scheduler_info()["workers"])
    if nb_workers >= 2:
        break
    time.sleep(1)
print(nb_workers)

28


In [5]:
!squeue -u albert7a

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
           8152700     hsw24 make_pro albert7a PD       0:00      1 (Priority)
           8152701     hsw24 make_pro albert7a PD       0:00      1 (Priority)
           8152702     hsw24 make_pro albert7a PD       0:00      1 (Priority)
           8152703     hsw24 make_pro albert7a PD       0:00      1 (Priority)
           8152431     hsw24 make_pro albert7a  R      19:06      1 n2160
           8152432     hsw24 make_pro albert7a  R      19:06      1 n2829
           8152433     hsw24 make_pro albert7a  R      19:06      1 n2831
           8152434     hsw24 make_pro albert7a  R      19:06      1 n2832
           8152435     hsw24 make_pro albert7a  R      19:06      1 n2833
           8152436     hsw24 make_pro albert7a  R      19:06      1 n2835
           8152437     hsw24 make_pro albert7a  R      19:06      1 n2209
           8152699     hsw24 make_pro albert7a  R       0:53      1 n2897


In [6]:
import sys
import numpy as np
import numpy.ma as ma
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import cartopy.feature as cfeature
import cmocean
import glob

%matplotlib inline


/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/xarray/core/merge.py:16: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  PANDAS_TYPES = (pd.Series, pd.DataFrame, pd.Panel)


In [7]:
#params

plot_eke_monthly=False
plot_eke_12_month=True
plot_eke_annual=False


In [8]:
config='NACHOS12.L75'
case='MAA4001'
freq='1d'
year='2010'

datadir='/scratch/cnt0024/hmg2840/albert7a/'+str(config)+'/'+str(config)+'-'+str(case)+'-S/'+str(freq)+'/'+str(year)+'/'
#!ls $datadir
gridfile='/scratch/cnt0024/hmg2840/albert7a/NACHOS12.L75/NACHOS12.L75-I/NACHOS12.L75_coordinates.nc'
maskfile='/scratch/cnt0024/hmg2840/albert7a/NACHOS12.L75/NACHOS12.L75-I/NACHOS12.L75_MAA40_mesh_mask.nc'

In [9]:
dsgrid=xr.open_dataset(gridfile)
dsmask=xr.open_dataset(maskfile)

In [10]:
def compute_EKE(u2,v2,u2mean,v2mean):
    ua=0.5*((-1*u2+u2mean)+(-1*u2.shift(x=1)+u2mean.shift(x=1)))
    va=0.5*((-1*v2+v2mean)+(-1*v2.shift(y=1)+v2mean.shift(y=1)))
    uat=ua.rename({'depthu':'deptht'})
    vat=va.rename({'depthv':'deptht'})
    eke=0.5*(uat+vat)
    return eke


In [16]:
month_name={1:'January',2:'February',3:'March',4:'April',5:'May',6:'June',7:'July',8:'August',9:'September',10:'October',11:'November',12:'December'}
def plot_one_month(var,mask,vmin,vmax,cmap,title,figfile,month,year):
    fig, ax = plt.subplots(1,1,figsize=(10,15))
    var.where(mask==1).plot(cmap=cmap,ax=ax,vmin=vmin,vmax=vmax,add_labels=True)
    ax.set_title(month_name[month])
    fig.suptitle('Monthly mean of '+str(title))
    plt.savefig('Plots/'+config+'/Maps/EU/'+config+'-'+case+'y'+str(year)+'m'+str(month)+'_'+figfile+'.png')

In [ ]:
filesU=sorted(glob.glob(datadir+'/'+str(config)+'-'+str(case)+'_y'+str(year)+'m??d??.'+str(freq)+'_gridU.nc'))
filesV=sorted(glob.glob(datadir+'/'+str(config)+'-'+str(case)+'_y'+str(year)+'m??d??.'+str(freq)+'_gridV.nc'))
dsU=xr.open_mfdataset(filesU, chunks={'x':700,'y':1000,'time_counter':10,'depthu':1})
dsV=xr.open_mfdataset(filesV, chunks={'x':700,'y':1000,'time_counter':10,'depthv':1})
u=dsU.vozocrtx[:,:,651:1224,970:1213]
v=dsV.vomecrty[:,:,651:1224,970:1213]
mask=dsmask.tmask[0,:,651:1224,970:1213]
u2=u*u
v2=v*v
u2mean=u2.mean(dim='time_counter')
v2mean=v2.mean(dim='time_counter')
umonth=u.groupby('time_counter.month').mean('time_counter')
vmonth=v.groupby('time_counter.month').mean('time_counter')
umonth2=umonth*umonth
vmonth2=vmonth*vmonth
umonth2mean=umonth2.mean(dim='month')
vmonth2mean=vmonth2.mean(dim='month')
eke=compute_EKE(umonth2,vmonth2,umonth2mean,vmonth2mean)
for m in np.arange(1,13):
    print('Plotting for month '+str(m))
    plot_one_month(10000*eke[m-1,0],mask[0],-2500,2500,'cmo.balance','Surface EKE y'+year,'eke_surf',m,year)
   